In [0]:
!pip freeze | grep tensorflow==

In [0]:
!wget https://dft-datasets.s3.us-east-2.amazonaws.com/catalog_images.zip
!mkdir data
!unzip -q catalog_images.zip -d data

In [0]:
!ls data/train/Blusas/ | wc -l

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.resnet50 import ResNet50 
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_prep
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_prep
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [0]:
def experiment(transfer_learning, nn, preprocess_input):
  if transfer_learning:
    base_model=nn(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #imports the mobilenet model and discards the last 1000 neuron layer.
  else:
    base_model=nn(weights=None, include_top=False, input_shape=(224, 224, 3))

  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
  x=Dense(1024, activation='relu')(x) #dense layer 2
  x=Dense(512, activation='relu')(x) #dense layer 3
  preds=Dense(10, activation='softmax')(x) #final layer with softmax activation
  
  
  model=Model(inputs=base_model.input,outputs=preds)
  
  if transfer_learning:
    for layer in model.layers[:20]:
        layer.trainable=False
    for layer in model.layers[20:]:
        layer.trainable=True
  
  train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

  train_generator=train_datagen.flow_from_directory('./data/train/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=16,
                                                 class_mode='categorical',
                                                 shuffle=True)
  eval_generator=train_datagen.flow_from_directory('./data/eval/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=16,
                                                 class_mode='categorical',
                                                 shuffle=True)
  
  model.compile(optimizer=Adam(lr=0.00002),loss='categorical_crossentropy',metrics=['accuracy'])
  # Adam optimizer
  # loss function will be categorical cross entropy
  # evaluation metric will be accuracy

  step_size_train=train_generator.n//train_generator.batch_size
  step_size_eval=eval_generator.n//eval_generator.batch_size
  model.fit_generator(generator=train_generator,
                     steps_per_epoch=step_size_train,
                     validation_data=eval_generator, 
                     validation_steps=step_size_eval,
                     epochs=5, verbose=1)

In [0]:
experiment(False, MobileNet, mobile_prep)
experiment(True, MobileNet, mobile_prep)

In [0]:
experiment(False, ResNet50, resnet_prep)
experiment(True, ResNet50, resnet_prep)